In [465]:
import datetime as dt
from dateutil.relativedelta import relativedelta

start = dt.datetime.now() - relativedelta(years=5)
end = dt.datetime.now()
days = 7
period = 14

In [466]:
import pandas_datareader as pdr

btc = pdr.DataReader('BTC-USD', 'yahoo', start, end)

btc = btc.drop(btc.index[[i for i in range(len(btc.T.columns)) if i % days != 0]])

In [467]:
import pandas_ta as pta

btc["Rsi"] = pta.rsi(btc.Close, length=period)
btc["Ema"] = pta.ema(btc["Rsi"], length=period)


buying_spots = [[btc.index[i], btc.Close[i]] for i, v in enumerate(btc.Rsi) if btc.Ema[i] < btc.Rsi[i]]

buying_spots_x = [a[0] for a in buying_spots]
buying_spots_y = [a[1] for a in buying_spots]

In [468]:
import plotly.offline as py
import plotly.graph_objs as go

py.init_notebook_mode(connected=True)

py.iplot(
    go.Figure(
        data=[
            go.Candlestick(x=btc.index, open=btc.Open, high=btc.High, low=btc.Low, close=btc.Close), 
            go.Scatter(x=buying_spots_x, y=buying_spots_y, mode='markers', marker=dict(size=5, color='blue'))
        ],
        layout=go.Layout(title='Bitcoin Candlestick with Range Slider', yaxis={'type': 'log'}, xaxis={'rangeslider':{'visible':False}})), 
    filename='bitcoin_candlestick')


import plotly.express as px

px.line(btc, y=["Rsi", "Ema"], title="Bitcoin RSI and EMA").show()